In [1]:
import re
import numpy as np
import json
import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import keras
import random
import joblib

2025-04-12 12:54:44.177506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
ps.stem?

In [4]:
ps = PorterStemmer()

def preprocess_text(text):

    words = nltk.word_tokenize(text)
#     words = [ps.stem(word.lower()) for word in words if word.isalnum()]  # Apply stemming
    return words

In [77]:
def create_bow(sentence, vocab):
#     sentence_words = preprocess_text(sentence)
    words = nltk.word_tokenize(sentence)
    words = [ps.stem(w.lower()) for w in words if w.isalnum()]
    sentence_words = sorted(list(set(words)))
    bow = [1 if word in sentence_words else 0 for word in vocab]
    return np.array(bow)

In [78]:
with open("/Users/apple/Desktop/testing/intents.json") as file:
    data = json.load(file)

In [79]:
words =[]
for intents in data["intents"]:
    for patterns in intents["patterns"]:
        words.extend(preprocess_text(patterns))
       

In [80]:

words = [ps.stem(w.lower()) for w in words if w.isalnum()]
words = sorted(list(set(words)))
words = np.asarray(words)

In [81]:
words

array(['a', 'about', 'afternoon', 'aka', 'alien', 'anim', 'anyon',
       'appreci', 'are', 'asdkjd', 'asl', 'avail', 'bake', 'banana',
       'been', 'beethoven', 'berri', 'best', 'bizarr', 'bla', 'blorp',
       'blue', 'boopiti', 'bop', 'bot', 'buy', 'bye', 'cake', 'can',
       'capac', 'car', 'care', 'casablanca', 'cat', 'catch', 'cereal',
       'choic', 'cloud', 'color', 'conspiraci', 'cost', 'could', 'd',
       'daft', 'dealership', 'dee', 'detail', 'dingl', 'dinosaur',
       'disappear', 'display', 'do', 'doe', 'dollar', 'dolphin', 'doodl',
       'dream', 'engin', 'even', 'exist', 'fast', 'fastest', 'favorit',
       'featur', 'find', 'fit', 'flame', 'flibberti', 'floo', 'for',
       'fraz', 'frizzl', 'froo', 'fruit', 'get', 'give', 'gloobiti',
       'gloop', 'go', 'good', 'goodby', 'greet', 'ha', 'happi', 'have',
       'hello', 'help', 'here', 'hey', 'hi', 'higgledi', 'how', 'howdi',
       'i', 'if', 'in', 'india', 'invent', 'is', 'it', 'jabber', 'jibber',
       'jugg

In [82]:
X=[]
y=[]
for intents in data["intents"]:
    for patterns in intents["patterns"]:
        X.append(create_bow(patterns,words))
        y.append(intents["tag"])

In [83]:
X = np.asarray(X)
y = np.asarray(y)

In [84]:
y

array(['greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye',
       'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye',
       'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks',
       'thanks', 'thanks', 'thanks', 'thanks', 'show_cars', 'show_cars',
       'show_cars', 'show_cars', 'show_cars', 'show_cars', 'show_cars',
       'show_cars', 'show_cars', 'show_cars', 'buy_car', 'buy_car',
       'buy_car', 'buy_car', 'buy_car', 'buy_car', 'buy_car', 'buy_car',
       'buy_car', 'buy_car', 'car_price', 'car_price', 'car_price',
       'car_price', 'car_price', 'car_price', 'car_price', 'car_price',
       'car_price', 'car_details', 'car_details', 'car_details',
       'car_details', 'car_details', 'car_details', 'car_details',
       'car_details', 'car_

In [85]:
classes = ['greeting','goodbye','thanks','show_cars','buy_car','car_price','car_details','car_speed','garbage']

In [86]:
Y=[]
for word in y:
    enc = [1 if w in word else 0 for w in classes]
    Y.append(enc)

In [87]:
Y = np.asarray(Y)
Y.shape

(140, 9)

In [88]:
X.shape

(140, 225)

In [89]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,shuffle=True,test_size=0.2)

In [90]:
X_train.shape

(112, 225)

In [91]:
y_train.shape

(112, 9)

In [92]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(len(X_train[0]),), activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
#     keras.layers.Dense(32, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(16, activation='relu'),
#     keras.layers.Dropout(0.5),
    keras.layers.Dense(len(y_train[0]), activation='softmax')  # Output layer matches number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1)


Epoch 1/100
23/23 [==============================] - 1s 2ms/step - loss: 2.2156 - accuracy: 0.1339 
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 2.0959 - accuracy: 0.3125
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 2.0000 - accuracy: 0.3661
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 1.8973 - accuracy: 0.4107
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 1.7773 - accuracy: 0.4018
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 1.6996 - accuracy: 0.3929
Epoch 7/100
23/23 [==============================] - 0s 2ms/step - loss: 1.5827 - accuracy: 0.4196
Epoch 8/100
23/23 [==============================] - 0s 2ms/step - loss: 1.4990 - accuracy: 0.4464
Epoch 9/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3596 - accuracy: 0.5179
Epoch 10/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3284 - accuracy: 0.5268
Epoch 11

23/23 [==============================] - 0s 3ms/step - loss: 0.0302 - accuracy: 1.0000
Epoch 84/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0293 - accuracy: 1.0000
Epoch 85/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 86/100
23/23 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 87/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 1.0000
Epoch 88/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 89/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0171 - accuracy: 0.9911
Epoch 90/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 91/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0244 - accuracy: 1.0000
Epoch 92/100
23/23 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 93/100


In [94]:
joblib.dump(model,'model2.h5')

INFO:tensorflow:Assets written to: ram://67bc872c-77c7-4032-97a4-4c4904742d40/assets


['model2.h5']

In [95]:
model = joblib.load('model2.h5')

In [96]:
pred = model.predict(X_test)

1/1 [==============================] - 0s 64ms/step


In [97]:
final = [ np.argmax(p) for p in pred]
final

[8,
 8,
 7,
 0,
 0,
 3,
 5,
 8,
 8,
 8,
 4,
 2,
 8,
 8,
 8,
 8,
 2,
 0,
 2,
 8,
 1,
 8,
 8,
 3,
 8,
 7,
 2,
 0]

In [98]:
y_test

array([[0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0,

In [99]:
def extract_car_name(user_input):
    # Tokenize and lower the input
    tokens = user_input.lower().split()
    
    # Check for matching tokens in known car names
    for car in car_details:
        car_tokens = car.split()
        if all(token in tokens for token in car_tokens):
            return car
    return None

def get_car_details(car_name):
    car_name = car_name.lower()
    if car_name in car_details:
        return car_details[car_name]
    return None



In [100]:
for car in car_details:
    print(car)

tesla model s
bmw 3 series
audi a8
mercedes-benz e-class
ford mustang
chevrolet corvette
porsche 911
honda civic
toyota camry
nissan gt-r
lamborghini huracan
ferrari 488
mazda cx-5
hyundai kona
kia stinger


In [101]:
with open("/Users/apple/Desktop/testing/dataset.json") as file:
    car_details = json.load(file)

In [102]:
def preprocess_input(user_input):
    # Tokenize and stem the user input
    user_input = nltk.word_tokenize(user_input)
    user_input = [ps.stem(word.lower()) for word in user_input]
    
    # Create a bag-of-words vector
    bag = [0] * len(words)
    for w in user_input:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array([bag])

def classify_intent(user_input):
    # Preprocess the input and predict the intent
    bow = preprocess_input(user_input)
    res = model.predict(bow)[0]
    error_threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > error_threshold]

    # Sort by probability
    results.sort(key=lambda x: x[1], reverse=True)
    return results

def get_response(intents_list, intents_json,user_input):
    tag = intents_list[0][0]
    intent_tag = classes[tag]
    
    # Find the corresponding intent and select a random response
    if intent_tag == "car_price":
        car = extract_car_name(user_input)
        return random.choice(data['intents'][tag]['responses'])+car_details[car]['price']
    elif intent_tag == "buy_car":
        car = extract_car_name(user_input)
        return random.choice(data['intents'][tag]['responses'])+car_details[car]['location']
    elif intent_tag == "car_details":
        car = extract_car_name(user_input)
        return random.choice(data['intents'][tag]['responses'])+car_details[car]['details']
    elif intent_tag == "car_speed":
        car = extract_car_name(user_input)
        return random.choice(data['intents'][tag]['responses'])+car_details[car]['speed']
    elif intent_tag == "garbage":
#         car = extract_car_name(user_input)
        return random.choice(data['intents'][tag]['responses'])
    elif intent_tag == "show_cars":
        detail = ""
        for car in car_details:
            detail+=car+"\n"
        return detail
    
        
        
    for intent in intents_json['intents']:
        if intent['tag'] == intent_tag:
            return random.choice(intent['responses'])

def chatbot_response(user_input):
    intents_list = classify_intent(user_input)
    response = get_response(intents_list, data,user_input)
    return response

In [103]:
data['intents'][tag]['responses']

NameError: name 'tag' is not defined

In [104]:
for car in car_details:
    print(car)

tesla model s
bmw 3 series
audi a8
mercedes-benz e-class
ford mustang
chevrolet corvette
porsche 911
honda civic
toyota camry
nissan gt-r
lamborghini huracan
ferrari 488
mazda cx-5
hyundai kona
kia stinger


In [107]:
print("Chatbot is running! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    response = chatbot_response(user_input)
    print(f"Bot: {response}")

Chatbot is running! Type 'quit' to exit.
You: show me some cars;
1/1 [==============================] - 0s 33ms/step
Bot: tesla model s
bmw 3 series
audi a8
mercedes-benz e-class
ford mustang
chevrolet corvette
porsche 911
honda civic
toyota camry
nissan gt-r
lamborghini huracan
ferrari 488
mazda cx-5
hyundai kona
kia stinger

You: what is the price of Mazda cx-5
1/1 [==============================] - 0s 26ms/step
Bot: This car is priced at $26,700
You: quit


In [128]:
intents_list = classify_intent("show me some cars")
intents_list

1/1 [==============================] - 0s 23ms/step


[[3, 0.99988866]]

In [131]:
classes[3]

'show_cars'

In [138]:
words=[]
word_list = nltk.word_tokenize("what is the top speed of this car?")
words.extend(word_list)
words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
words = sorted(list(set(words)))
words

['car', 'is', 'of', 'speed', 'the', 'thi', 'top', 'what']

In [75]:
stemmer = PorterStemmer()

# Initialize lists for storing training data
words = []
classes = []
documents = []

# Loop through each intent in the JSON file
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Add to documents with its associated tag (intent)
        documents.append((word_list, intent['tag']))
        
    # Add intent to the classes list
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Stem and lower each word, and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

print(f"Words: {words}")
print(f"Classes: {classes}")

# Prepare training data
training = []
output_empty = [0] * len(classes)

# Create bag-of-words for each pattern
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # Create the bag-of-words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Output is a one-hot encoded list for each intent
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Shuffle the data and convert it into numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

# Split the features (X) and labels (y)
X_train = np.array(list(training[:, 0]))
y_train = np.array(list(training[:, 1]))

print(f"Training Data: {X_train}")

Words: ['about', 'appreci', 'buy', 'bye', 'can', 'car', 'cost', 'detail', 'do', 'doe', 'even', 'fast', 'give', 'good', 'goodby', 'have', 'hello', 'hey', 'hi', 'how', 'i', 'is', 'it', 'later', 'list', 'me', 'more', 'morn', 'much', 'of', 'price', 'purchas', 's', 'see', 'show', 'some', 'specif', 'speed', 'tell', 'thank', 'the', 'thi', 'top', 'what', 'where', 'you']
Classes: ['buy_car', 'car_details', 'car_price', 'car_speed', 'goodbye', 'greeting', 'show_cars', 'thanks']
Training Data: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 1 0 ... 0 0 0]]


In [76]:
print(model.predict(X_train[3:4]))
print(y_train[3:4])

1/1 [==============================] - 0s 31ms/step
[[0.043018   0.03909636 0.01554782 0.05664859 0.22830929 0.6123372
  0.00238561 0.00265713]]
[[0 0 0 1 0 0 0 0]]


In [77]:
np.argmax(model.predict(X_train[3:4]))

1/1 [==============================] - 0s 23ms/step


5

In [78]:
X_train.shape

(26, 46)

In [79]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(len(X_train[0]),), activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(y_train[0]), activation='softmax')  # Output layer matches number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=5, verbose=1)


Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 2.1192 - accuracy: 0.0769
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0634 - accuracy: 0.1538
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 2.0334 - accuracy: 0.0769
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0045 - accuracy: 0.3846
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.9827 - accuracy: 0.2308
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9956 - accuracy: 0.1154
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0095 - accuracy: 0.1538
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 2.0053 - accuracy: 0.0769
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9894 - accuracy: 0.1154
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8524 - accuracy: 0.3846
Epoch 11/200
6/6 [===========

6/6 [==============================] - 0s 2ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0760 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0690 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0384 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0602 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 176/200
6/6 [==============================] - 0s 2ms/step - loss: 0.0702 - accuracy: 1.0000
Epoch 177/200
6/6 [=====

In [80]:
def preprocess_input(user_input):
    # Tokenize and stem the user input
    user_input = nltk.word_tokenize(user_input)
    user_input = [ps.stem(word.lower()) for word in user_input]
    
    # Create a bag-of-words vector
    bag = [0] * len(words)
    for w in user_input:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array([bag])

def classify_intent(user_input):
    # Preprocess the input and predict the intent
    bow = preprocess_input(user_input)
    res = model.predict(bow)[0]
    error_threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > error_threshold]

    # Sort by probability
    results.sort(key=lambda x: x[1], reverse=True)
    return results

def get_response(intents_list, intents_json):
    tag = intents_list[0][0]
    intent_tag = classes[tag]
    
    # Find the corresponding intent and select a random response
    for intent in intents_json['intents']:
        if intent['tag'] == intent_tag:
            return random.choice(intent['responses'])

def chatbot_response(user_input):
    intents_list = classify_intent(user_input)
    response = get_response(intents_list, data)
    return response

In [81]:
print("Chatbot is running! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    response = chatbot_response(user_input)
    print(f"Bot: {response}")

Chatbot is running! Type 'quit' to exit.
You: what
1/1 [==============================] - 0s 76ms/step
Bot: Hello! How can I assist you today?
You: huh
1/1 [==============================] - 0s 25ms/step
Bot: Hi there! What car would you like to know about?
You: lamborghini
1/1 [==============================] - 0s 27ms/step
Bot: Hi there! What car would you like to know about?
You: I want to see some cars today
1/1 [==============================] - 0s 26ms/step
Bot: Here are a few options: [list of car names]. Which one would you like more details on?
You: choose one for me
1/1 [==============================] - 0s 31ms/step
Bot: This car has a speed of [speed], costs [price], and is available at [location]. Would you like to know more?
You: quit


In [76]:
# Example dataset of intents and BoW vectors
# X = [...]  # List of BoW vectors
# y = [...]  # Corresponding intent labels

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a simple MLP classifier
classifier = MLPClassifier(hidden_layer_sizes=(8, 8), max_iter=1000)
classifier.fit(X_train, y_train)



# Test the classifier
accuracy = classifier.score(X_test, y_test)
print(f"Model accuracy: {accuracy}")

Model accuracy: 0.16666666666666666


In [114]:
X_train[0].shape

(46,)

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Train a simple MLP classifier
model = keras.models.Sequential([
    keras.layers.Input(shape=(46,)),
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dense(64,activation="relu"),
#     keras.layers.Dense(32,activation="relu"),
    keras.layers.Dense(8,activation="softmax")
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Test the classifier
model.fit(X_train,y_train,epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 421ms/step - loss: 2.1152 - accuracy: 0.0500
Epoch 2/50
1/1 [==============================] - 0s 4ms/step - loss: 2.0811 - accuracy: 0.1000
Epoch 3/50
1/1 [==============================] - 0s 4ms/step - loss: 2.0484 - accuracy: 0.1500
Epoch 4/50
1/1 [==============================] - 0s 4ms/step - loss: 2.0173 - accuracy: 0.2000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 1.9873 - accuracy: 0.3500
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: 1.9582 - accuracy: 0.4000
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 1.9301 - accuracy: 0.4500
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 1.9029 - accuracy: 0.6500
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 1.8760 - accuracy: 0.7000
Epoch 10/50
1/1 [==============================] - 0s 6ms/step - loss: 1.8490 - accuracy: 0.7500
Epoch 11/50
1/1 [====================

In [235]:
model.evaluate(X_test)

1/1 [==============================] - 0s 92ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]